In [1]:
! pip install pymorphy2  

In [2]:
import nltk   # добавляем все нужные библиотеки
import random
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Базовая модель бота

In [71]:
BOT_CONFIG = {  # пробная база знаний, откуда берутся значения
    'intents': {
        'hello': {
            'examples': ['Привет',"Хелоу","Здавствуй"],
            'responces': ['Прив',"Хай","Алоха"]
        },
        'bye': {
            'examples': ['Пока',"Гудбай","Удачи"],
            'responces': ['До свиданья',"До скорой встречи","Пока"]
        },
        'helps': {
            'examples': ['Команды'],
            'responces': ['1) Добавить задачу \n 2) Открыть список задач \n 3) Редактировать задачу \n 4) Удалить задачу']
        },
        'start': {
            'examples': ['Старт'],
            'responces': ['Привет, меня зовут Кринжер, я твой персональный таскменеджер. Вот что я умею: \n 1) Добавить задачу \n 2) Открыть список задач \n 3) Редактировать задачу \n 4) Удалить задачу \n P. S.  Полный список команд ты можешь увидеть по запросу Команды']
        },
        'add_task': {
            'examples': ['1','Добать задачу', '1) Добавить задачу', ' Добавить'],
            'responces': ['Введите задачу:']
        }
    }
}

In [64]:
def clean (text):  # очистка входа от мусора (регистр, знаки)
    text = text.lower()
    cleaned_text = ''
    for char in text:
        if char in '@абвгдеёжзийклмнопрстуфхцчшщъыьэюя ':
            cleaned_text += char
    return cleaned_text

In [65]:
def get_intent(in_text):   # распознавание введеных данных
    for intent in BOT_CONFIG['intents'].keys():
        for example in BOT_CONFIG['intents'][intent]['examples']:
            text1 =  clean(example) 
            text2 = clean(in_text)
            if nltk.edit_distance(text1, text2)/max(len(text1),len(text2)) <  0.4 :
                return intent
    return 'Не удалось определить интент'

In [66]:
def bot(in_text):  # бот отвечает на сообщеньки
    intent = get_intent(in_text)
    if intent == 'Не удалось определить интент':
        return 'Извини, ничего не понял'
    else:
        return random.choice(BOT_CONFIG['intents'][intent]['responces'])

In [74]:
in_text = ''     #  запуск бота, с возможностью выхода
while in_text != 'Выход':
    in_text = input()
    out_text = bot(in_text)
    print(out_text)

страт
Извини, ничего не понял
старт
Привет, меня зовут Кринжер, я твой персональный таскменеджер. Вот что я умею: 
 1) Добавить задачу 
 2) Открыть список задач 
 3) Редактировать задачу 
 4) Удалить задачу 
 P. S.  Полный список команд ты можешь увидеть по запросу Команды
добавь
Введите задачу:
пока
До скорой встречи
выход
Извини, ничего не понял
Выход
Извини, ничего не понял


# Подключение к телеграму

In [17]:
# ! pip install python-telegram-bot --upgrade --user

In [8]:
import logging   # подключаем библиотеки телеграма для запуска
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

def start(update: Update, context: CallbackContext) -> None:
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),)

def help_command(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Help!')

def echo(update: Update, context: CallbackContext) -> None:
    update.message.reply_text(bot(update.message.text))

def main() -> None:
    updater = Updater("1830028553:AAGELjUoBd6QaImIk4YvuyIHxmX5zl1Iya4")
    dispatcher = updater.dispatcher
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))
    updater.start_polling()
    updater.idle()

In [68]:
main()  # запускаем бота в телеграм


2022-01-07 13:40:45,809 - apscheduler.scheduler - INFO - Scheduler started
2022-01-07 13:41:30,047 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2022-01-07 13:41:30,050 - apscheduler.scheduler - INFO - Scheduler has been shut down
